In [8]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam

# Imports everything
# Visualization tools
import torchvision
import torchvision.transforms.v2 as transforms
import torchvision.transforms.functional as F
import matplotlib.pyplot as plt

#Runs process on GPU that NVDIA Provided 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.is_available()

#Automatically sets data to training and validating sets
train_set = torchvision.datasets.MNIST("./data/", train=True, download=True)
valid_set = torchvision.datasets.MNIST("./data/", train=False, download=True)


#Converts data set to tensors
trans = transforms.Compose([transforms.ToTensor()])
train_set.transform = trans
valid_set.transform = trans

#Loads data into bateches, ideal for Deep Learning
batch_size = 32
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_set, batch_size=batch_size)

# Overall this code seems to break from the typical norm of setting a X_test, X_valid and y_test, Y-valid by just taking batches of the data set as a whole.  

#Since this data is suppose to classify numbers from 0-9, the n_classes is suppose to represent the output layer and options it has to chosse correct layer. 
n_classes = 10
input_size = 1 * 28 * 28

layers = [
    nn.Flatten(), # Flatten model to 1-D array for ease and processing
    nn.Linear(input_size, 512),  # Linear Regression
    nn.ReLU(),  # ReLU activation
    nn.Linear(512, 512),  # Hidden
    nn.ReLU(),  # ReLU activation
    nn.Linear(512, n_classes)  # Output
]

model = nn.Sequential(*layers) # Creates sequential model
model.to(device)
model = torch.compile(model)

loss_function = nn.CrossEntropyLoss() # loss function for classification
optimizer = Adam(model.parameters()) # Optimizer for adapting learning rate

train_N = len(train_loader.dataset)
valid_N = len(valid_loader.dataset)

# takes node with highest value and assumes that is what the model says is the classification of the image.
# remeniscient of Pandas with methods called
def get_batch_accuracy(output, y, N):
    pred = output.argmax(dim=1, keepdim=True)
    correct = pred.eq(y.view_as(pred)).sum().item()
    return correct / N

def train():
    loss = 0
    accuracy = 0

    model.train()
    for x, y in train_loader:
        x, y = x.to(device), y.to(device)
        output = model(x)
        optimizer.zero_grad()
        batch_loss = loss_function(output, y)
        batch_loss.backward()
        optimizer.step()

        loss += batch_loss.item()
        accuracy += get_batch_accuracy(output, y, train_N)
    print('Train - Loss: {:.4f} Accuracy: {:.4f}'.format(loss, accuracy))
    
def validate():
    loss = 0
    accuracy = 0

    model.eval()
    with torch.no_grad():
        for x, y in valid_loader:
            x, y = x.to(device), y.to(device)
            output = model(x) # Similar to model.predict(X_test) for machine learning

            loss += loss_function(output, y).item()
            accuracy += get_batch_accuracy(output, y, valid_N)
    print('Valid - Loss: {:.4f} Accuracy: {:.4f}'.format(loss, accuracy))

epochs = 5

for epoch in range(epochs):
    print('Epoch: {}'.format(epoch))
    train()
    validate()




/usr/local/lib/python3.10/dist-packages/torchvision/transforms/v2/_deprecated.py:41: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.
  warnings.warn(


Epoch: 0
Train - Loss: 380.0457 Accuracy: 0.9387
Valid - Loss: 34.3750 Accuracy: 0.9669
Epoch: 1
Train - Loss: 157.1377 Accuracy: 0.9740
Valid - Loss: 27.2710 Accuracy: 0.9733
Epoch: 2
Train - Loss: 109.2711 Accuracy: 0.9819
Valid - Loss: 22.9953 Accuracy: 0.9770
Epoch: 3
Train - Loss: 83.8851 Accuracy: 0.9857
Valid - Loss: 26.8590 Accuracy: 0.9766
Epoch: 4
Train - Loss: 69.6284 Accuracy: 0.9885
Valid - Loss: 25.2077 Accuracy: 0.9787


tensor([[5]], device='cuda:0')